In [1]:
import glob
import pandas as pd
import seaborn as sb
import numpy as np
from sklearn.model_selection import train_test_split

In [20]:
ampscz_scores = pd.read_csv("AMPSCZ_scores.csv")
ampscz_scores["sub-ses"] = (
    ampscz_scores["sub_id_gs"].astype(str)
    + "-"
    + ampscz_scores["ses_id_gs"].astype(str)
)
ampscz_scores["scores"] = (
    ampscz_scores["scores"].replace("-", "-1").fillna("-1").astype(int)
)
ampscz_scores["comment"] = ampscz_scores["comment"].fillna("")
ampscz_scores["site"] = ampscz_scores["sub_id_gs"].str[0:2]
ampscz_scores=ampscz_scores[ampscz_scores["scores"]>-1]
ampscz_scores

,sub_id_gs,ses_id_gs,timepoint_gs,scores,comment,sub-ses,site
0,BI02450,202306231,month_2_arm_1,2,"severe ringing, ghosting at eyes, motion",BI02450-202306231,BI
1,BI02450,202304111,baseline_arm_1,3,ringing throughout,BI02450-202304111,BI
3,BI05529,202306051,baseline_arm_2,4,no notable issues,BI05529-202306051,BI
5,BI05652,202304171,baseline_arm_1,3,some ringing/motion in temporal poles,BI05652-202304171,BI
7,BI05874,202305181,baseline_arm_1,3,Some ringing,BI05874-202305181,BI
...,...,...,...,...,...,...,...
1113,YA20849,202308041,baseline_arm_1,3,ringing,YA20849-202308041,YA
1115,YA23168,202308211,baseline_arm_1,4,,YA23168-202308211,YA
1117,YA23568,202308311,baseline_arm_1,3,Minor ringing,YA23568-202308311,YA
1119,YA24934,202308281,baseline_arm_1,4,,YA24934-202308281,YA


In [21]:
finetune_df = pd.read_csv("/home/chbricout/Master/mrart/src/dataset/ampscz/finetune.csv", index_col=0)
pretrain_df = pd.read_csv("/home/chbricout/Master/mrart/src/dataset/ampscz/pretrain.csv", index_col=0)
site_df = pd.read_csv("/home/chbricout/Master/mrart/src/dataset/ampscz/sites.csv", index_col=0)
pretrain_site = site_df[site_df['group']=='pretrain']["site"]
pretrain_df

,sub_id_gs,ses_id_gs,timepoint_gs,score,comment,sub-ses,site,motion,group,data
3,BI05529,202306051,baseline_arm_2,4,no notable issues,BI05529-202306051,BI,False,train,AMPSCZ-Preproc/subjects/sub-BI05529/ses-202306...
18,BM13122,202308041,baseline_arm_1,4,"a bit grainy, otherwise fine",BM13122-202308041,BM,False,train,AMPSCZ-Preproc/subjects/sub-BM13122/ses-202308...
21,BM44608,202307181,month_2_arm_1,4,NaN,BM44608-202307181,BM,False,train,AMPSCZ-Preproc/subjects/sub-BM44608/ses-202307...
40,BM99192,202304191,month_2_arm_2,4,NaN,BM99192-202304191,BM,False,train,AMPSCZ-Preproc/subjects/sub-BM99192/ses-202304...
50,CA04203,202304191,baseline_arm_2,4,no notable issues,CA04203-202304191,CA,False,train,AMPSCZ-Preproc/subjects/sub-CA04203/ses-202304...
...,...,...,...,...,...,...,...,...,...,...
1009,TE00307,202210201,baseline_arm_1,4,A little blurry,TE00307-202210201,TE,False,test,AMPSCZ-Preproc/subjects/sub-TE00307/ses-202210...
1010,TE00307,202212151,month_2_arm_1,4,NaN,TE00307-202212151,TE,False,test,AMPSCZ-Preproc/subjects/sub-TE00307/ses-202212...
1036,TE16009,202305311,baseline_arm_1,4,No notable issues,TE16009-202305311,TE,False,test,AMPSCZ-Preproc/subjects/sub-TE16009/ses-202305...
1037,TE16009,202307261,month_2_arm_1,4,no notable issues,TE16009-202307261,TE,False,test,AMPSCZ-Preproc/subjects/sub-TE16009/ses-202307...


In [17]:
data_in_pretrain_site = ampscz_scores[ampscz_scores['site'].isin(pretrain_site)]
data_in_pretrain_site

,sub_id_gs,ses_id_gs,timepoint_gs,scores,comment,sub-ses,site
0,BI02450,202306231,month_2_arm_1,2,"severe ringing, ghosting at eyes, motion",BI02450-202306231,BI
1,BI02450,202304111,baseline_arm_1,3,ringing throughout,BI02450-202304111,BI
3,BI05529,202306051,baseline_arm_2,4,no notable issues,BI05529-202306051,BI
5,BI05652,202304171,baseline_arm_1,3,some ringing/motion in temporal poles,BI05652-202304171,BI
7,BI05874,202305181,baseline_arm_1,3,Some ringing,BI05874-202305181,BI
...,...,...,...,...,...,...,...
1064,WU09592,202212131,baseline_arm_1,4,No notable issues,WU09592-202212131,WU
1065,WU09592,202303301,month_2_arm_1,4,,WU09592-202303301,WU
1066,WU10549,202304051,baseline_arm_1,4,,WU10549-202304051,WU
1067,WU11737,202306291,baseline_arm_1,4,no notable issues,WU11737-202306291,WU


In [26]:
not_used_pretrain = data_in_pretrain_site[-data_in_pretrain_site['sub-ses'].isin(pretrain_df['sub-ses'])]
not_used_pretrain

,sub_id_gs,ses_id_gs,timepoint_gs,scores,comment,sub-ses,site
0,BI02450,202306231,month_2_arm_1,2,"severe ringing, ghosting at eyes, motion",BI02450-202306231,BI
1,BI02450,202304111,baseline_arm_1,3,ringing throughout,BI02450-202304111,BI
5,BI05652,202304171,baseline_arm_1,3,some ringing/motion in temporal poles,BI05652-202304171,BI
7,BI05874,202305181,baseline_arm_1,3,Some ringing,BI05874-202305181,BI
11,BI08676,202306141,baseline_arm_1,4,Negligible ringing,BI08676-202306141,BI
...,...,...,...,...,...,...,...
1034,TE15483,202305091,baseline_arm_1,3,faint ringing,TE15483-202305091,TE
1052,WU01590,202212011,month_2_arm_1,3,moderate ringing,WU01590-202212011,WU
1054,WU04342,202212191,month_2_arm_2,3,motion throughout,WU04342-202212191,WU
1059,WU06851,202212191,month_2_arm_1,4,slight ringing but not significant,WU06851-202212191,WU
